In [1]:
import json
import os
import math
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import tensorflow as tf

C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

In [3]:
#Variable definitions
#signal-Signal S
#frame_size -divided frames
#hop_length-size intersection between frames
#P- given parameter
#defined function- Input = signal; Output = t_new point   (Step 1-4 in the Paper)
def amplitude_envelope(signal,sample_duration, frame_size=1024, hop_length=512,P=0.35):
    """Calculate the amplitude envelope of a signal with a given frame size nad hop length."""
    amplitude_envelope = []  #Define amplitude envolope list
    index_signal_amplitude_envelope=[]  #Define index for amplitude envelope
    # calculate amplitude envelope for each frame
    for i in range(0, len(signal), hop_length): 
        amplitude_envelope_current_frame = max(signal[i:i+frame_size]) 
        amplitude_envelope.append(amplitude_envelope_current_frame)
        #append index for each amplitude envelope obtained in each frame
        index_signal_amplitude_envelope.append(i+signal[i:i+frame_size].tolist().index(max(signal[i:i+frame_size].tolist())))
    
    ae_signal=np.array(amplitude_envelope)                        #list of amplitude envelope
    t=np.array(index_signal_amplitude_envelope)*sample_duration   #corresponding time for amplitude envelope
    t_max=t[ae_signal.tolist().index(max(ae_signal))]       #t_new=time for max amplitude - 0.35   #0.35 is a parameter
    
    if t_max<=P:
        t_new=0
    else:
        t_new=t_max-P
    return t_new

In [4]:
def save_stft(dataset_path, n_fft, hop_length):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.
        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "labels": [],  # syllables/segments
        "mfcc": []   # STFTs...,mels,.....MFCCs
    }
   
    j=0
    a=0
    p=0
    #Section 4.1.1 Signal length module
    # loop through all sub-folder
    
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # process all audio files
        for f in filenames:
            # save DIGIT label  in the mapping
            semantic_label = f.split("_")[0]
            # Save file path with source dir path
            file_path = os.path.join(dirpath, f)
             # input file wav file to obtain signal and new sample rate
            signal, sr = librosa.load(file_path, sr=SAMPLE_RATE) #duration=DURATION
             # New output file name
            trimmed_path="recordings-github\\"+f
             # Display
            print(trimmed_path)
            # Calculate duration in seconds for the input Signal
            dur=len(signal)/SAMPLE_RATE
            sample_duration = 1 / sr
            
            if dur>0.74:
                #print(dur)
                p+=1
             # Step 1-5
            if len(signal)>= num_expected_samples:   #Step 5 checking for trimming
                offset=amplitude_envelope(signal,sample_duration)   #Step 1-4
                trim_signal, sr = librosa.load(file_path,sr=SAMPLE_RATE,offset=offset,duration=DURATION)   #trimming
                padded_signal=trim_signal
                
                if len(trim_signal)<num_expected_samples:
                    num_missing_samples = num_expected_samples - len(trim_signal)
                    padded_signal = np.pad(trim_signal,
                                          (0, num_missing_samples),
                                          mode="constant")   #Padding
                    a+=1
                    
            elif len(signal) < num_expected_samples:   #Step 5 checking for padding
                num_missing_samples = num_expected_samples - len(signal)
                padded_signal = np.pad(signal,
                                      (0, num_missing_samples),
                                      mode="constant")       #Padding
                
            #librosa.output.write_wav(trimmed_path, padded_signal, sr)
            
            # Section 4.1.2 Feature extraction module
            # Input new signal ; Output MFCCs
            # New signal=padded_signal; sr=sample rate; n_mfcc=no of mfcc coefficients; n_fft=interval for fft; hop_length=sliding window
            mfcc = librosa.feature.mfcc(padded_signal, sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
            
            # Display mfcc dimension 
            print(mfcc.shape)
            
            # normalizing
            S_min=np.min(mfcc)
            S_max=np.max(mfcc)
            X_norm_mfcc=(mfcc - S_min)/(S_max - S_min)
            
            # append normalized mfccs features and labels to each data containers
            data["labels"].append(semantic_label)
            data["mfcc"].append(X_norm_mfcc.T.tolist())
            #data["mfcc"].append(mfcc.T.tolist())
        
            j+=1


    # save MFCCs and its labels to json file
    with open(JSON_PATH, "w") as fp:
        json.dump(data, fp, indent=4)

In [5]:
# Defined parameters
DATASET_PATH = "recordings"   # English source folder download from https://github.com/Jakobovski/free-spoken-digit-dataset
JSON_PATH =  "MNIST-digit-en-mfcc-16.json"  #destination json file name
SAMPLE_RATE = 22050 #22050 #   8000
n_fft=512       #511  358
hop_length=256   #123  296
DURATION= 0.74    #0.74
num_expected_samples=int(DURATION*SAMPLE_RATE)
num_mfcc=16

In [6]:
if __name__ == "__main__":
    save_stft(DATASET_PATH,n_fft, hop_length)   # run save_mfcc function


Processing: recordings
recordings-github\0_george_0.wav
(16, 64)
0
recordings-github\0_george_1.wav
(16, 64)
0
recordings-github\0_george_10.wav
(16, 64)
1
recordings-github\0_george_11.wav
(16, 64)
1
recordings-github\0_george_12.wav
(16, 64)
1
recordings-github\0_george_13.wav
(16, 64)
1
recordings-github\0_george_14.wav
(16, 64)
1
recordings-github\0_george_15.wav
(16, 64)
1
recordings-github\0_george_16.wav
(16, 64)
1
recordings-github\0_george_17.wav
(16, 64)
1
recordings-github\0_george_18.wav
(16, 64)
1
recordings-github\0_george_19.wav
(16, 64)
1
recordings-github\0_george_2.wav
(16, 64)
1
recordings-github\0_george_20.wav
(16, 64)
1
recordings-github\0_george_21.wav
(16, 64)
1
recordings-github\0_george_22.wav
(16, 64)
1
recordings-github\0_george_23.wav
(16, 64)
1
recordings-github\0_george_24.wav
(16, 64)
1
recordings-github\0_george_25.wav
(16, 64)
1
recordings-github\0_george_26.wav
(16, 64)
1
recordings-github\0_george_27.wav
(16, 64)
1
recordings-github\0_george_28.wav


recordings-github\0_nicolas_4.wav
(16, 64)
1
recordings-github\0_nicolas_40.wav
(16, 64)
1
recordings-github\0_nicolas_41.wav
(16, 64)
1
recordings-github\0_nicolas_42.wav
(16, 64)
1
recordings-github\0_nicolas_43.wav
(16, 64)
1
recordings-github\0_nicolas_44.wav
(16, 64)
1
recordings-github\0_nicolas_45.wav
(16, 64)
1
recordings-github\0_nicolas_46.wav
(16, 64)
1
recordings-github\0_nicolas_47.wav
(16, 64)
1
recordings-github\0_nicolas_48.wav
(16, 64)
1
recordings-github\0_nicolas_49.wav
(16, 64)
1
recordings-github\0_nicolas_5.wav
(16, 64)
1
recordings-github\0_nicolas_6.wav
(16, 64)
1
recordings-github\0_nicolas_7.wav
(16, 64)
1
recordings-github\0_nicolas_8.wav
(16, 64)
1
recordings-github\0_nicolas_9.wav
(16, 64)
1
recordings-github\0_theo_0.wav
(16, 64)
1
recordings-github\0_theo_1.wav
(16, 64)
1
recordings-github\0_theo_10.wav
(16, 64)
1
recordings-github\0_theo_11.wav
(16, 64)
1
recordings-github\0_theo_12.wav
(16, 64)
1
recordings-github\0_theo_13.wav
(16, 64)
1
recordings-git

recordings-github\1_jackson_26.wav
(16, 64)
1
recordings-github\1_jackson_27.wav
(16, 64)
1
recordings-github\1_jackson_28.wav
(16, 64)
1
recordings-github\1_jackson_29.wav
(16, 64)
1
recordings-github\1_jackson_3.wav
(16, 64)
1
recordings-github\1_jackson_30.wav
(16, 64)
1
recordings-github\1_jackson_31.wav
(16, 64)
1
recordings-github\1_jackson_32.wav
(16, 64)
1
recordings-github\1_jackson_33.wav
(16, 64)
1
recordings-github\1_jackson_34.wav
(16, 64)
1
recordings-github\1_jackson_35.wav
(16, 64)
1
recordings-github\1_jackson_36.wav
(16, 64)
1
recordings-github\1_jackson_37.wav
(16, 64)
1
recordings-github\1_jackson_38.wav
(16, 64)
1
recordings-github\1_jackson_39.wav
(16, 64)
1
recordings-github\1_jackson_4.wav
(16, 64)
1
recordings-github\1_jackson_40.wav
(16, 64)
1
recordings-github\1_jackson_41.wav
(16, 64)
1
recordings-github\1_jackson_42.wav
(16, 64)
1
recordings-github\1_jackson_43.wav
(16, 64)
1
recordings-github\1_jackson_44.wav
(16, 64)
1
recordings-github\1_jackson_45.wav
(

(16, 64)
1
recordings-github\1_yweweler_12.wav
(16, 64)
1
recordings-github\1_yweweler_13.wav
(16, 64)
1
recordings-github\1_yweweler_14.wav
(16, 64)
1
recordings-github\1_yweweler_15.wav
(16, 64)
1
recordings-github\1_yweweler_16.wav
(16, 64)
1
recordings-github\1_yweweler_17.wav
(16, 64)
1
recordings-github\1_yweweler_18.wav
(16, 64)
1
recordings-github\1_yweweler_19.wav
(16, 64)
1
recordings-github\1_yweweler_2.wav
(16, 64)
1
recordings-github\1_yweweler_20.wav
(16, 64)
1
recordings-github\1_yweweler_21.wav
(16, 64)
1
recordings-github\1_yweweler_22.wav
(16, 64)
1
recordings-github\1_yweweler_23.wav
(16, 64)
1
recordings-github\1_yweweler_24.wav
(16, 64)
1
recordings-github\1_yweweler_25.wav
(16, 64)
1
recordings-github\1_yweweler_26.wav
(16, 64)
1
recordings-github\1_yweweler_27.wav
(16, 64)
1
recordings-github\1_yweweler_28.wav
(16, 64)
1
recordings-github\1_yweweler_29.wav
(16, 64)
1
recordings-github\1_yweweler_3.wav
(16, 64)
1
recordings-github\1_yweweler_30.wav
(16, 64)
1
reco

recordings-github\2_lucas_41.wav
(16, 64)
1
recordings-github\2_lucas_42.wav
(16, 64)
1
recordings-github\2_lucas_43.wav
(16, 64)
1
recordings-github\2_lucas_44.wav
(16, 64)
1
recordings-github\2_lucas_45.wav
(16, 64)
1
recordings-github\2_lucas_46.wav
(16, 64)
1
recordings-github\2_lucas_47.wav
(16, 64)
1
recordings-github\2_lucas_48.wav
(16, 64)
1
recordings-github\2_lucas_49.wav
(16, 64)
1
recordings-github\2_lucas_5.wav
(16, 64)
1
recordings-github\2_lucas_6.wav
(16, 64)
1
recordings-github\2_lucas_7.wav
(16, 64)
1
recordings-github\2_lucas_8.wav
(16, 64)
1
recordings-github\2_lucas_9.wav
(16, 64)
1
recordings-github\2_nicolas_0.wav
(16, 64)
1
recordings-github\2_nicolas_1.wav
(16, 64)
1
recordings-github\2_nicolas_10.wav
(16, 64)
1
recordings-github\2_nicolas_11.wav
(16, 64)
1
recordings-github\2_nicolas_12.wav
(16, 64)
1
recordings-github\2_nicolas_13.wav
(16, 64)
1
recordings-github\2_nicolas_14.wav
(16, 64)
1
recordings-github\2_nicolas_15.wav
(16, 64)
1
recordings-github\2_nic

...

recordings-github\7_lucas_19.wav
(16, 64)
6
recordings-github\7_lucas_2.wav
(16, 64)
6
recordings-github\7_lucas_20.wav
(16, 64)
6
recordings-github\7_lucas_21.wav
(16, 64)
6
recordings-github\7_lucas_22.wav
(16, 64)
6
recordings-github\7_lucas_23.wav
(16, 64)
6
recordings-github\7_lucas_24.wav
(16, 64)
6
recordings-github\7_lucas_25.wav
(16, 64)
6
recordings-github\7_lucas_26.wav
(16, 64)
6
recordings-github\7_lucas_27.wav
(16, 64)
6
recordings-github\7_lucas_28.wav
(16, 64)
6
recordings-github\7_lucas_29.wav
(16, 64)
6
recordings-github\7_lucas_3.wav
(16, 64)
6
recordings-github\7_lucas_30.wav
(16, 64)
6
recordings-github\7_lucas_31.wav
(16, 64)
6
recordings-github\7_lucas_32.wav
(16, 64)
6
recordings-github\7_lucas_33.wav
(16, 64)
6
recordings-github\7_lucas_34.wav
(16, 64)
6
recordings-github\7_lucas_35.wav
(16, 64)
6
recordings-github\7_lucas_36.wav
(16, 64)
6
recordings-github\7_lucas_37.wav
(16, 64)
6
recordings-github\7_lucas_38.wav
(16, 64)
6
recordings-github\7_lucas_39.wav
(

recordings-github\7_yweweler_8.wav
(16, 64)
8
recordings-github\7_yweweler_9.wav
(16, 64)
8
recordings-github\8_george_0.wav
(16, 64)
8
recordings-github\8_george_1.wav
(16, 64)
8
recordings-github\8_george_10.wav
(16, 64)
8
recordings-github\8_george_11.wav
(16, 64)
8
recordings-github\8_george_12.wav
(16, 64)
8
recordings-github\8_george_13.wav
(16, 64)
8
recordings-github\8_george_14.wav
(16, 64)
8
recordings-github\8_george_15.wav
(16, 64)
8
recordings-github\8_george_16.wav
(16, 64)
8
recordings-github\8_george_17.wav
(16, 64)
8
recordings-github\8_george_18.wav
(16, 64)
8
recordings-github\8_george_19.wav
(16, 64)
8
recordings-github\8_george_2.wav
(16, 64)
8
recordings-github\8_george_20.wav
(16, 64)
8
recordings-github\8_george_21.wav
(16, 64)
8
recordings-github\8_george_22.wav
(16, 64)
8
recordings-github\8_george_23.wav
(16, 64)
8
recordings-github\8_george_24.wav
(16, 64)
8
recordings-github\8_george_25.wav
(16, 64)
8
recordings-github\8_george_26.wav
(16, 64)
8
recordings-

recordings-github\8_nicolas_37.wav
(16, 64)
8
recordings-github\8_nicolas_38.wav
(16, 64)
8
recordings-github\8_nicolas_39.wav
(16, 64)
8
recordings-github\8_nicolas_4.wav
(16, 64)
8
recordings-github\8_nicolas_40.wav
(16, 64)
8
recordings-github\8_nicolas_41.wav
(16, 64)
8
recordings-github\8_nicolas_42.wav
(16, 64)
8
recordings-github\8_nicolas_43.wav
(16, 64)
8
recordings-github\8_nicolas_44.wav
(16, 64)
8
recordings-github\8_nicolas_45.wav
(16, 64)
8
recordings-github\8_nicolas_46.wav
(16, 64)
8
recordings-github\8_nicolas_47.wav
(16, 64)
8
recordings-github\8_nicolas_48.wav
(16, 64)
8
recordings-github\8_nicolas_49.wav
(16, 64)
8
recordings-github\8_nicolas_5.wav
(16, 64)
8
recordings-github\8_nicolas_6.wav
(16, 64)
8
recordings-github\8_nicolas_7.wav
(16, 64)
8
recordings-github\8_nicolas_8.wav
(16, 64)
8
recordings-github\8_nicolas_9.wav
(16, 64)
8
recordings-github\8_theo_0.wav
(16, 64)
8
recordings-github\8_theo_1.wav
(16, 64)
8
recordings-github\8_theo_10.wav
(16, 64)
8
recor

recordings-github\9_jackson_22.wav
(16, 64)
8
recordings-github\9_jackson_23.wav
(16, 64)
8
recordings-github\9_jackson_24.wav
(16, 64)
8
recordings-github\9_jackson_25.wav
(16, 64)
8
recordings-github\9_jackson_26.wav
(16, 64)
8
recordings-github\9_jackson_27.wav
(16, 64)
8
recordings-github\9_jackson_28.wav
(16, 64)
8
recordings-github\9_jackson_29.wav
(16, 64)
8
recordings-github\9_jackson_3.wav
(16, 64)
8
recordings-github\9_jackson_30.wav
(16, 64)
8
recordings-github\9_jackson_31.wav
(16, 64)
8
recordings-github\9_jackson_32.wav
(16, 64)
8
recordings-github\9_jackson_33.wav
(16, 64)
8
recordings-github\9_jackson_34.wav
(16, 64)
8
recordings-github\9_jackson_35.wav
(16, 64)
8
recordings-github\9_jackson_36.wav
(16, 64)
8
recordings-github\9_jackson_37.wav
(16, 64)
8
recordings-github\9_jackson_38.wav
(16, 64)
8
recordings-github\9_jackson_39.wav
(16, 64)
8
recordings-github\9_jackson_4.wav
(16, 64)
8
recordings-github\9_jackson_40.wav
(16, 64)
8
recordings-github\9_jackson_41.wav
(

8
recordings-github\9_yweweler_11.wav
(16, 64)
8
recordings-github\9_yweweler_12.wav
(16, 64)
8
recordings-github\9_yweweler_13.wav
(16, 64)
8
recordings-github\9_yweweler_14.wav
(16, 64)
8
recordings-github\9_yweweler_15.wav
(16, 64)
8
recordings-github\9_yweweler_16.wav
(16, 64)
8
recordings-github\9_yweweler_17.wav
(16, 64)
8
recordings-github\9_yweweler_18.wav
(16, 64)
8
recordings-github\9_yweweler_19.wav
(16, 64)
8
recordings-github\9_yweweler_2.wav
(16, 64)
8
recordings-github\9_yweweler_20.wav
(16, 64)
8
recordings-github\9_yweweler_21.wav
(16, 64)
8
recordings-github\9_yweweler_22.wav
(16, 64)
8
recordings-github\9_yweweler_23.wav
(16, 64)
8
recordings-github\9_yweweler_24.wav
(16, 64)
8
recordings-github\9_yweweler_25.wav
(16, 64)
8
recordings-github\9_yweweler_26.wav
(16, 64)
8
recordings-github\9_yweweler_27.wav
(16, 64)
8
recordings-github\9_yweweler_28.wav
(16, 64)
8
recordings-github\9_yweweler_29.wav
(16, 64)
8
recordings-github\9_yweweler_3.wav
(16, 64)
8
recordings-gi